

## Objectif:

  Récupération des données sur le web avec le scraping 
 

## Source:
   
   Récupération des données de France sur le site https://www.journaldunet.com/

## Methodes:

  Utilisation des librairies Python pour le scraping web (bs4,selenium,requestes ....)
 
 ## Resultas:

  Fichiers csv des ville de France leur démographie, Chômage en France pour chaque ville
   
  






In [1]:
#librairies nécessaire
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
from pandas import DataFrame
from multiprocessing import Pool
import time

# Scraping 
   - ## Villes de France et leurs liens 

In [2]:
# colonnes de notre data set
colonnes=['ville','lien']

In [ ]:
# creation d'un data frame vide pour enrigstrer les ville et les liens 
tableau=DataFrame(columns=colonnes)
# sauvgarder Data set vide en csv
tableau.to_csv('C:/Users/lamine/PycharmProjects/dataset/lienVille.csv',index=False)
dico={}
dico['ville']=''
dico['lien']=''
# lien pour recuprer les lien de la ville
url="http://www.journaldunet.com/management/ville/index/villes?page="

In [ ]:
# Ouverture du data set avec with open avec l'encoding utf-8
with open('C:/Users/lamine/PycharmProjects/dataset/lienVille.csv','a',encoding = "utf-8") as csvfile:
    # Ecriture des données recuperer dans un dictionnaire
    writer = csv.DictWriter(csvfile, fieldnames=colonnes, lineterminator='\n')
    # Parcourir toutes les page du site
    for numeroPage in range(1,701):
        print('le numero de page en traitement est : '+ str(numeroPage))
        # se connecter au l'url du site 
        req = requests.get(url+str(numeroPage))
        # recuperer le contenu html du la page 
        contenu = req.content
        soup = bs(contenu, 'html.parser')
        tousLeslines = soup.findAll('a')
        # remplissage du dico
        for lien in tousLeslines:
            if '/ville-'in lien['href'] and lien['href'][:11]=='/management':
                dico['lien']=lien['href']
                dico['ville']=lien.text
                # ecriture par ligne sur le dictinnaire 
                writer.writerow(dico)


- ## Informmations sur les villes de france 

In [ ]:
# Colonnes de notre dataframe
colonnes = ['ville', 'lien', 'Région', 'Département',
                'Etablissement public de coopération intercommunale (EPCI)',
                'Code postal (CP)','Code Insee',
                'Nom des habitants',
                'Population (2017)',
                'Population : rang national (2017)',
                'Densité de population (2017)',
                'Taux de chômage (2016)',
                'Pavillon bleu',
                "Ville d'art et d'histoire",
                'Ville fleurie',
                'Ville internet',
                'Superficie (surface)',
                'Altitude min.',
                'Altitude max.',
                'Latitude',
                'Longitude' ]


#le fichier des inforation existe ou pas
if os.path.isfile('C:/Users/lamine/PycharmProjects/dashboard1/dataset/info.csv'):

    tableauInfos=pd.read_csv('C:/Users/lamine/PycharmProjects/dashboard1/dataset/info.csv',error_bad_lines=False, dtype='unicode')
    tableauLiens=pd.read_csv('C:/Users/lamine/PycharmProjects/dashboard1/dataset/lienVille.csv')
    
else:

    # creation de notre dataframe info
    tableauInfos = DataFrame(columns=colonnes)
    tableauInfos.to_csv('C:/Users/lamine/PycharmProjects/dashboard1/dataset/info.csv', index=False)
    # recupere la liste des liens 
    tableauLiens=pd.read_csv('C:/Users/lamine/PycharmProjects/dashboard1/dataset/lienVille.csv')
    listeLiens = tableauLiens['lien']
# verefication des liens 
listLiens=[i for i in listeLiens if str(i[:11])=='/management']



def parse(lien):
    #initialisation d'un dictionnaire
    dico = {i: '' for i in colonnes}
    colonne=dico.keys()
    # url du site (site + lien pour la ville)
    req = requests.get("http://www.journaldunet.com" + lien)
   
    # pose de 2 seconde apres l'exucution des ligne d'avant
    time.sleep(4)
    if req.status_code==200: # satuts code =200 c'est il s'est conecter à la page ou pas (404 non exist .....)
        with open ('C:/Users/lamine/PycharmProjects/dashboard1/dataset/info.csv','a',encoding = "utf-8") as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=colonne,lineterminator ='\n')

            contenu=req.content
            soup=bs(contenu,'html.parser')
            tables = soup.findAll('table', class_='odTable odTableAuto')

            dico['lien'] = lien
            dico['ville'] = tableauLiens[tableauLiens['lien'] == lien]['ville'].iloc[0]

            for i in range(len(tables)):
                tousLesTr = tables[i].findAll('tr')
                for tr in tousLesTr[1:]:
                    cle = tr.findAll('td')[0].text
                    valeur = tr.findAll('td')[1].text

                    if "Nom des habitants" in cle:
                        dico["Nom des habitants"] = str(valeur)
                    elif "Taux de chômage" in cle:
                        dico["Taux de chômage (2016)"] = str(valeur)
                    else:
                        dico[cle] = str(valeur)
            time.sleep(1)
            writer.writerow(dico)
            print(dico)

if __name__ == "__main__":
    # Multiprocessing avec la fonction Pool
    with Pool(15) as p:
        p.map(parse, listeLiens)